In [10]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit = 120,
    memory_key = "chat_history", # 프롬프트의 {chat_history}와 연결
    return_messages = True # 메시지 객체(리스트) 형태로 결과를 반환하도록 설정
)

# MessagesPlaceholder: 메모리에서 가져온 메시지 리스트가 들어갈 위치 지정
# 'chat_history' 변수명을 메모리의 momory_key와 일치시켜서 주입
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("human", "{question}"),
])

# LLMChain: 모델, 메모리, 프롬프트를 하나로 묶어 실행 과정을 자동화
chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt = prompt,
    verbose=True
)

chain.predict(question = "My name is Yujin")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Yujin

> Finished chain.


'Hello Yujin! How can I assist you today?'

In [12]:
chain.predict(question = "I live in Seongnam")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Yujin
AI: Hello Yujin! How can I assist you today?
Human: I live in Seongnam

> Finished chain.


"That's great to know, Yujin! Is there anything specific you would like to know or talk about regarding Seongnam?"

In [13]:
chain.predict(question = "What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Yujin
AI: Hello Yujin! How can I assist you today?
Human: I live in Seongnam
AI: That's great to know, Yujin! Is there anything specific you would like to know or talk about regarding Seongnam?
Human: What is my name?

> Finished chain.


'Your name is Yujin.'

In [14]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='My name is Yujin'),
  AIMessage(content='Hello Yujin! How can I assist you today?'),
  HumanMessage(content='I live in Seongnam'),
  AIMessage(content="That's great to know, Yujin! Is there anything specific you would like to know or talk about regarding Seongnam?"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Yujin.')]}